In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import sys
import os

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics
from scipy import stats

reload(sys)
#sys.setdefaultencoding("ISO-8859-1")
sys.setdefaultencoding("UTF-8")

In [2]:
def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [3]:
def clean_all_text(allText, numLines):
    clean_train_data = []
    for i in xrange(0, numLines):
        clean_train_data.append(clean_text(allText[i]))
    return clean_train_data

In [4]:
def featureSelection(train_x, task, train_y):
    rows, cols = train_x.shape
    top_info_words_numbers = [100, 200, 300, 500, 700, 1000, 2000, 5000, 7000, 8000, 9000, 10000, cols-1]
    top_info_words_numbers =  sorted(top_info_words_numbers, reverse=True)

    feature_selection_result = {}
    
    task_to_filenames = {'age': ['age-important-words-using-info-gain.txt', 'age-important-words-using-gain-ratio.txt'],
                 'gender': ['gender-important-words-info-gain.txt', 'gender-important-words-gain-ratio.txt']
                }
    
    filenames = task_to_filenames[task]
    for filename in filenames:
        with open(filename) as f:
            alist = [line.rstrip() for line in f]
        all_indices_ranked = alist[0].split(',')
        all_indices_ranked = [int(x) for x in all_indices_ranked]
        all_indices_ranked = [x-1 for x in all_indices_ranked]
        
        list_of_scores = []
        for num_info_words in top_info_words_numbers:
            clf = svm.SVC(kernel='linear', C=1)
            scoring_function = 'accuracy'
            
            xx = [all_indices_ranked[x] for x in range(0, num_info_words)]
            xx = tuple(xx)
            smaller_train_x = train_x[:, xx]

            scores = cross_validation.cross_val_score(clf, smaller_train_x, train_y, cv=10, scoring=scoring_function)
            list_of_scores.append(scores)
            
            feature_selection_result[filename] = list_of_scores
    return feature_selection_result
            

In [5]:
def doSVMwithPoly():
    pass

In [6]:
def doSVMwithRBF():
    pass

In [7]:
def calculatePValue(input_dictionary):
    p_values_dictionary = {}
    for each_key in input_dictionary.keys():
        list_of_scores = input_dictionary[each_key]
        i = range(0,len(list_of_scores))
        list_of_pvalues = []
        for x, i  in zip(list_of_scores,i):
            z_stat, p_val = stats.ranksums(list_of_scores[0], x)
            list_of_pvalues.append( p_val)
        p_values_dictionary[each_key] = list_of_pvalues
    return p_values_dictionary
        

In [8]:
datafile = "summary-english-truth.txt"

train = pd.read_csv(datafile, header=0, delimiter="\t", quoting=1)

clean_train_data = clean_all_text(train["text"], train["text"].size)

vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None)
train_x = vectorizer.fit_transform(clean_train_data)
train_x = train_x.toarray()

task = "age"
train_y = train[task]
feature_selection_result = featureSelection(train_x, task, train_y)
p_values_dictionary = calculatePValue(feature_selection_result)
#    doSVMwithPoly()
#    doSVMwithRBF()

#    doRandomForest()
#    doBoosting()

In [9]:
feature_selection_result['age-important-words-using-info-gain.txt'][0].mean()

0.69124999999999992

In [10]:
p_values_dictionary

{'age-important-words-using-gain-ratio.txt': [1.0,
  0.075661572143887038,
  0.075661572143887038,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.00015705228423075119],
 'age-important-words-using-info-gain.txt': [1.0,
  0.075661572143887038,
  0.075661572143887038,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102,
  0.021133928129161102]}